In [13]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from xgboost import XGBClassifier
import os
import joblib
import datetime
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon  # Polygon 클래스 임포트 추가
import warnings


# 데이터 경로 설정
DATA_PATH = '/content/drive/MyDrive/25년 해군 AI 경진대회/combined_ais_20230601_20240531.csv'
ZONE_DIR = '/content/drive/MyDrive/25년 해군 AI 경진대회/dataset/구역 데이터'
MODEL_DIR = '/content/drive/MyDrive/25년 해군 AI 경진대회/model_result'
os.makedirs(MODEL_DIR, exist_ok=True)

zone_files = [
    ('Area_Navy_train.csv', 'navy_train'),
    ('Area_Near_Sea.csv', 'near_sea'),
    ('Area_Restrict_zone.csv', 'restrict'),
    ('Area_Sea_cable_lv1_poly.csv', 'sea_cable'),
    ('Area_Special_restrict_zone_poly.csv', 'special_restrict'),
    ('Area_Target_Area.csv', 'target_area')
]

In [16]:
def load_zone_csv(filepath, label):
    """
    CSV 파일에서 폴리곤 데이터를 로드하여 GeoDataFrame으로 반환
    """
    try:
        # CSV 파일 읽기
        df = pd.read_csv(filepath)
        print(f"{filepath}의 열 이름: {list(df.columns)}")

        # 열 이름을 소문자로 변환하여 'lon'과 'lat' 열 찾기
        df.columns = [col.lower() for col in df.columns]
        if 'lon' not in df.columns or 'lat' not in df.columns:
            warnings.warn(f"{filepath}: 'lon' 또는 'lat' 열을 찾을 수 없습니다.")
            return gpd.GeoDataFrame()

        # 좌표 리스트 생성
        coords = list(zip(df['lon'], df['lat']))
        print(f"{filepath}의 좌표 데이터: {coords[:5]} ... (총 {len(coords)}개)")  # 디버깅용

        # 중복된 점 제거
        coords = [coords[i] for i in range(len(coords)) if i == 0 or coords[i] != coords[i-1]]

        # 최소 3개의 점 필요
        if len(coords) < 3:
            warnings.warn(f"{filepath}: 폴리곤 생성에 필요한 최소 점(3개) 미달")
            return gpd.GeoDataFrame()

        # 폴리곤 폐쇄 여부 확인
        if coords[0] != coords[-1]:
            coords.append(coords[0])

        # 폴리곤 생성
        poly = Polygon(coords)

        # 유효성 검사 및 자가 교차 해결
        if not poly.is_valid:
            poly = poly.buffer(0)  # 자가 교차 해결
            if not poly.is_valid:
                warnings.warn(f"{filepath}: 유효하지 않은 폴리곤 생성 (buffer 후에도 유효하지 않음)")
                return gpd.GeoDataFrame()

        return gpd.GeoDataFrame([{'label': label}], geometry=[poly])

    except Exception as e:
        warnings.warn(f"{filepath} 처리 중 오류: {str(e)}")
        return gpd.GeoDataFrame()

In [17]:
# 메인 로직
print("1. 데이터 로드 및 전처리")
zone_gdfs = [load_zone_csv(os.path.join(ZONE_DIR, fname), lbl) for fname, lbl in zone_files]
valid_gdfs = [g for g in zone_gdfs if not g.empty]

if valid_gdfs:
    zones_gdf = pd.concat(valid_gdfs, ignore_index=True)
    print(f"Zone polygons loaded: {len(zones_gdf)}")
else:
    print("유효한 폴리곤 데이터가 없습니다. CSV 파일을 확인하세요.")

1. 데이터 로드 및 전처리
/content/drive/MyDrive/25년 해군 AI 경진대회/dataset/구역 데이터/Area_Navy_train.csv의 열 이름: ['OBJNUM', 'LON', 'LAT']
/content/drive/MyDrive/25년 해군 AI 경진대회/dataset/구역 데이터/Area_Navy_train.csv의 좌표 데이터: [(125.4508609, 34.55173509), (125.4508806, 34.5504108), (125.450875, 34.54908642), (125.450844, 34.54776227), (125.4507876, 34.5464387)] ... (총 1672개)
/content/drive/MyDrive/25년 해군 AI 경진대회/dataset/구역 데이터/Area_Near_Sea.csv의 열 이름: ['id', 'LON', 'LAT']
/content/drive/MyDrive/25년 해군 AI 경진대회/dataset/구역 데이터/Area_Near_Sea.csv의 좌표 데이터: [(126.8538328127415, 37.233167708126366), (126.44498963147878, 37.20243716773674), (126.64685708836554, 37.09569697152781), (126.89619507565028, 37.13067212613029), (126.8538328127415, 37.233167708126366)] ... (총 634개)
/content/drive/MyDrive/25년 해군 AI 경진대회/dataset/구역 데이터/Area_Restrict_zone.csv의 열 이름: ['OBJNUM', 'LON', 'LAT']
/content/drive/MyDrive/25년 해군 AI 경진대회/datase

In [21]:
print("zones_gdf의 현재 CRS:", zones_gdf.crs)

zones_gdf의 현재 CRS: None


In [22]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

# AIS 데이터 로드
ais_df = pd.read_csv(DATA_PATH)
bbox_polys = [box(min(lon1, lon2), min(lat1, lat2), max(lon1, lon2), max(lat1, lat2))
              for lon1, lon2, lat1, lat2 in zip(
                  ais_df['min_lon'], ais_df['max_lon'], ais_df['min_lat'], ais_df['max_lat'])]
ais_gdf = gpd.GeoDataFrame(ais_df, geometry=bbox_polys, crs='EPSG:4326')

# zones_gdf에 CRS 설정 (사용자가 정의했다고 가정)
import geopandas as gpd

# zones_gdf의 현재 CRS 확인
print("zones_gdf의 현재 CRS:", zones_gdf.crs)

# CRS가 없거나 EPSG:4326과 다른 경우 처리
if zones_gdf.crs is None or zones_gdf.crs != 'EPSG:4326':
    zones_gdf = zones_gdf.set_crs('EPSG:4326', allow_override=True)
else:
    zones_gdf = zones_gdf.to_crs('EPSG:4326')

# 공간 특징 생성
join_df = gpd.sjoin(ais_gdf[['geometry']], zones_gdf[['label', 'geometry']],
                    how='left', predicate='intersects').reset_index().rename(columns={'index': 'ais_idx'})
pivot = (join_df.dropna(subset=['label']).assign(hit=1)
         .pivot_table(index='ais_idx', columns='label', values='hit', aggfunc='sum', fill_value=0))
ais_gdf['num_zone_hits'] = pivot.sum(axis=1).reindex(ais_gdf.index, fill_value=0).astype(int)
for z in zones_gdf['label'].unique():
    ais_gdf[f'in_{z}'] = (pivot.get(z, 0) > 0).reindex(ais_gdf.index, fill_value=0).astype(int)

zones_gdf의 현재 CRS: None


In [23]:
# 2. 특징 엔지니어링
print("2. 특징 엔지니어링")
def engineer_features(df):
    df = df.copy()
    df['low_speed_zone_interaction'] = df['low_speed_ratio'] * df['restricted_zone_flag']
    df['ais_gap_interaction'] = df['off_events'] * (df['max_gap_sec'] / 3600)
    df['speed_stability'] = df['std_sog'] / (df['avg_sog'] + 0.1)
    df['sharp_turn_ratio'] = df['sharp_turns'] / (df['num_points'] + 1)
    df['non_korean_low_speed'] = (1 - df['is_korean_ship']) * df['low_speed_ratio']
    for col in df.select_dtypes(include=['object']).columns:
        if col == 'result':
            continue
        try:
            dt = pd.to_datetime(df[col], errors='raise')
            df[f'{col}_doy'] = dt.dt.dayofyear
            df[f'{col}_dow'] = dt.dt.weekday
            df.drop(columns=[col], inplace=True)
        except:
            df.drop(columns=[col], inplace=True)
    return df

X_full = engineer_features(ais_gdf.drop(columns=['geometry', 'MMSI']))

# result 열 처리
print("result 열의 고유 값:", X_full['result'].unique())
y = X_full['result'].map({'true': 1, 'false': 0, '1': 1, '0': 0}, na_action='ignore').fillna(0).astype(int)
X = X_full.drop(columns=['result'])

2. 특징 엔지니어링
result 열의 고유 값: [False  True]


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 로드
data = pd.read_csv("YOUR_DATA_PATH.csv")
y = data['result']

# 클래스 분포 확인
print("Original y 클래스 분포:", y.value_counts())

# 양성 클래스가 없는 경우 경고
if len(y.unique()) < 2:
    print("경고: y에 단일 클래스만 존재합니다. 모델 학습이 불가능합니다.")
    print("원본 데이터를 확인하고 양성 클래스를 포함한 데이터를 추가하세요.")
    exit()

# 데이터 분할
X = data.drop('result', axis=1)  # 특성 데이터
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# 분할 후 클래스 분포 확인
print("y_train 클래스 분포:", y_train.value_counts())
print("y_val 클래스 분포:", y_val.value_counts())
print("y_test 클래스 분포:", y_test.value_counts())

# 모델 학습 코드 (클래스 분포가 정상일 경우 진행)

FileNotFoundError: [Errno 2] No such file or directory: 'YOUR_DATA_PATH.csv'